# Read in the Raw Data, Load the YAML metadata, and Clean the Data

In [1]:
# Import the required modules

import pandas as pd
import numpy as np
import yaml

from sklearn import linear_model

from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [2]:
# Read in the raw CSV data
# https://www.fhfa.gov/sites/default/files/2024-06/nsmo_v50_1321_puf.csv

raw_df = pd.read_csv('nsmo_v50_1321_puf.csv')
raw_df.shape

(50542, 543)

In [3]:
# Load YAML files containing metadata into Python as dictionaries

# Load the labels of each variable into a dictionary
# https://www.fhfa.gov/sites/default/files/2024-06/nsmo_v50_labels.sas

with open('variable_labels.yaml', 'r') as file:
    variable_labels_dict = yaml.safe_load(file)
    
# Load the the format of each variable into a dictionary
# https://www.fhfa.gov/sites/default/files/2024-06/nsmo_v50_labels.sas

with open('variable_formats.yaml', 'r') as file:
    variable_formats_dict = yaml.safe_load(file)
    
# Load the categories for every categorical variable (exclude null categories) into a dictionary
# https://www.fhfa.gov/sites/default/files/2024-06/nsmo_v50_formats.sas

with open('categorical_variables_categories.yaml', 'r') as file:
    categorical_variables_categories_dict = yaml.safe_load(file)

In [4]:
# Clean data by converting negative values and "." values (representing missing values) into null values (i.e., NaN)

for col in raw_df.columns:
    # Exclude the Mortgage Performance Status variables because they have letters representing specific categories
    if variable_formats_dict[col] != 'PSTATFM':
        raw_df.loc[raw_df[col] < 0, col] = np.nan
        raw_df.loc[raw_df[col] == ".", col] = np.nan

In [5]:
# Check out a few obs after data cleaning

raw_df.tail()

,nsmoid,survey_wave,analysis_weight,x05a,x05b,x05c,x05d,x05e,x05f,x05g,...,mtmltv0621,mtmltv0921,mtmltv1221,mtmltv0322,mtmltv0622,mtmltv0922,mtmltv1222,mtmltv0323,mtmltv0623,mtmltv0923
50537,531289.0,34.0,2117.79,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,64.0,61.0,59.0,59.0,59.0,59.0,59.0,60.0
50538,546643.0,34.0,1738.92,3.0,3.0,2.0,2.0,2.0,1.0,3.0,...,NaN,NaN,79.0,77.0,74.0,72.0,72.0,71.0,71.0,71.0
50539,512993.0,34.0,2353.26,1.0,2.0,2.0,2.0,2.0,1.0,2.0,...,NaN,NaN,95.0,91.0,88.0,85.0,84.0,84.0,83.0,82.0
50540,518631.0,34.0,5283.75,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,NaN,NaN,56.0,53.0,50.0,49.0,49.0,49.0,48.0,48.0
50541,544740.0,34.0,1738.92,1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,NaN,NaN,80.0,74.0,69.0,66.0,65.0,64.0,63.0,63.0


In [6]:
# Create a set of all variable formats

variable_formats_set = set(variable_formats_dict.values())

In [7]:
# Create a list of the categorical variables and a list of the numeric variables

categorical_variables = []
numeric_variables = []

categorical_variable_formats = set(categorical_variables_categories_dict.keys())
numeric_variable_formats = variable_formats_set - categorical_variable_formats

for col in raw_df.columns:
    if variable_formats_dict[col] in categorical_variable_formats:
        categorical_variables.append(col)
    elif variable_formats_dict[col] in numeric_variable_formats:
        numeric_variables.append(col)
    else:
        print("Error in bifurcation")

In [8]:
raw_df.tail()

,nsmoid,survey_wave,analysis_weight,x05a,x05b,x05c,x05d,x05e,x05f,x05g,...,mtmltv0621,mtmltv0921,mtmltv1221,mtmltv0322,mtmltv0622,mtmltv0922,mtmltv1222,mtmltv0323,mtmltv0623,mtmltv0923
50537,531289.0,34.0,2117.79,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,64.0,61.0,59.0,59.0,59.0,59.0,59.0,60.0
50538,546643.0,34.0,1738.92,3.0,3.0,2.0,2.0,2.0,1.0,3.0,...,NaN,NaN,79.0,77.0,74.0,72.0,72.0,71.0,71.0,71.0
50539,512993.0,34.0,2353.26,1.0,2.0,2.0,2.0,2.0,1.0,2.0,...,NaN,NaN,95.0,91.0,88.0,85.0,84.0,84.0,83.0,82.0
50540,518631.0,34.0,5283.75,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,NaN,NaN,56.0,53.0,50.0,49.0,49.0,49.0,48.0,48.0
50541,544740.0,34.0,1738.92,1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,NaN,NaN,80.0,74.0,69.0,66.0,65.0,64.0,63.0,63.0


In [9]:
# Check out a few obs for just the categorical variable

raw_df[categorical_variables].tail()

,x05a,x05b,x05c,x05d,x05e,x05f,x05g,x06,x07,x08a,...,forb0621,forb0921,forb1221,forb0322,forb0622,forb0922,forb1222,forb0323,forb0623,forb0923
50537,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0,1.0,...,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0
50538,3.0,3.0,2.0,2.0,2.0,1.0,3.0,3.0,3.0,2.0,...,NaN,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0,2.0
50539,1.0,2.0,2.0,2.0,2.0,1.0,2.0,3.0,2.0,1.0,...,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0
50540,3.0,3.0,3.0,3.0,3.0,3.0,3.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0
50541,1.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,2.0,1.0,...,NaN,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [10]:
# Check out a few obs for just the numeric variable

raw_df[numeric_variables].tail()

,nsmoid,survey_wave,analysis_weight,x74r,rate_spread,pmms,term,ltv,cltv,dti,...,mtmltv0621,mtmltv0921,mtmltv1221,mtmltv0322,mtmltv0622,mtmltv0922,mtmltv1222,mtmltv0323,mtmltv0623,mtmltv0923
50537,531289.0,34.0,2117.79,57.0,0.64,3.11,40.0,64.0,64.0,42.0,...,NaN,NaN,64.0,61.0,59.0,59.0,59.0,59.0,59.0,60.0
50538,546643.0,34.0,1738.92,37.0,0.03,3.10,30.0,79.0,79.0,33.0,...,NaN,NaN,79.0,77.0,74.0,72.0,72.0,71.0,71.0,71.0
50539,512993.0,34.0,2353.26,26.0,NaN,3.10,30.0,95.0,95.0,35.0,...,NaN,NaN,95.0,91.0,88.0,85.0,84.0,84.0,83.0,82.0
50540,518631.0,34.0,5283.75,36.0,NaN,3.10,20.0,56.0,56.0,46.0,...,NaN,NaN,56.0,53.0,50.0,49.0,49.0,49.0,48.0,48.0
50541,544740.0,34.0,1738.92,42.0,0.08,3.05,30.0,80.0,80.0,20.0,...,NaN,NaN,80.0,74.0,69.0,66.0,65.0,64.0,63.0,63.0


In [11]:
# View survey answers for any given observation in a human readable format using the YAML metadata
# Deactivated code below because it will print 500+ lines because we have 500+ variables

if False:
    one_obs = raw_df.iloc[50541]
    # Loop through all columns for one obs
    for col, value in one_obs.items():
        # if it's a categorical variable, then look up the category
        if not(pd.isna(value)) and col in categorical_variables:
            print(variable_labels_dict[col], ":", categorical_variables_categories_dict[variable_formats_dict[col]][value])
        # else it's a numeric variable or null
        else:
            print(variable_labels_dict[col], ":", value)

# Process Data using Dummy Coding, Imputing, and Partitioning

In [12]:
# Create dummy variables for each category for each categorical variable

processed_df = pd.get_dummies(raw_df, columns=categorical_variables)

In [13]:
# Remove the ".0" in many of the dummy variable due to the columns in the raw data being floats

new_columns_list = []
for col in processed_df.columns:
    new_col = col.replace(".0", "")
    new_columns_list.append(new_col)
    
processed_df.columns = new_columns_list

In [14]:
# Retrieve the names of the new categorical variables (i.e., the dummy variables)

new_categorical_variables = []
for col in processed_df.columns:
    if col not in numeric_variables:
        new_categorical_variables.append(col)

In [15]:
# Check out a few obs for just the new categorical variable

processed_df[new_categorical_variables].tail()

,x05a_1,x05a_2,x05a_3,x05b_1,x05b_2,x05b_3,x05c_1,x05c_2,x05c_3,x05d_1,...,forb0922_1,forb0922_2,forb1222_1,forb1222_2,forb0323_1,forb0323_2,forb0623_1,forb0623_2,forb0923_1,forb0923_2
50537,1,0,0,1,0,0,1,0,0,1,...,0,1,0,1,0,1,0,1,0,1
50538,0,0,1,0,0,1,0,1,0,0,...,0,1,0,1,0,1,0,1,0,1
50539,1,0,0,0,1,0,0,1,0,0,...,0,1,0,1,0,1,0,1,0,1
50540,0,0,1,0,0,1,0,0,1,0,...,0,1,0,1,0,1,0,1,0,1
50541,1,0,0,1,0,0,1,0,0,1,...,0,1,0,1,0,1,0,1,0,1


In [16]:
# Impute missing values for all numeric variables using Mean Value Imputation
# The danger of this approach is that I am using all data (from both the training partition and the testing partition) to impute missing value
# This will cause "data leakage" because information from the holdout testing parition will "leak" into the imputation of the training partition

for col in processed_df.columns:
    if col in numeric_variables:
        processed_df[col] = processed_df[col].fillna(processed_df[col].mean())

In [17]:
# Identify the target variable

target_variable = 'rate_spread'

In [18]:
# List the NSMO variables to be excluded

list_of_NSMO_variables = ['nsmoid',         # NSMO Identification Number
                          'survey_wave',    # NSMO Survey Wave (Quarterly)
                          'analysis_weight' # NSMO Analysis Weight (Sampling Weight x Non-response Adjustment)
                         ]

In [19]:
# Create a compete list of all excluded variables, which includes the target variable and the NSMO variables

list_of_excluded_variables = list([target_variable]) + list_of_NSMO_variables
print(list_of_excluded_variables)

['rate_spread', 'nsmoid', 'survey_wave', 'analysis_weight']


In [20]:
# Segregate the predictor variables from the target variable

X = processed_df.drop(columns=list_of_excluded_variables)
y = processed_df[target_variable]

In [21]:
# Split the data into training and testing partitions

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80, random_state=0)

# Compare Supervised Learning Models: Gradient Boosting, Linear, LASSO, and Random Forest

In [22]:
# Fit a Gradient Boosting Regression model on the training partition, and then evaluate it on the testing partition
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingRegressor.html

params = {'max_depth': 4, 'learning_rate': 0.01, 'random_state':0}
grad_boost_reg = HistGradientBoostingRegressor(**params)
grad_boost_reg.fit(X_train, y_train)

mae = mean_absolute_error(y_test, grad_boost_reg.predict(X_test))
mse = mean_squared_error(y_test, grad_boost_reg.predict(X_test))
rmse = np.sqrt(mse)
grad_boost_reg_dict = {"model": "Gradient Boosting Regression", 
                       "Mean Absolute Error": mae, 
                       "Root Mean Squared Error": rmse}

In [23]:
# Fit a Linear Model on the training partition, and evaluate it on the testing partition
# There is a lot of Multicollinearity in this model because we are putting all predictor variables into the model
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

linear_reg = linear_model.LinearRegression()
linear_reg.fit(X_train,y_train)

mae = mean_absolute_error(y_test, linear_reg.predict(X_test))
mse = mean_squared_error(y_test, linear_reg.predict(X_test))
rmse = np.sqrt(mse)
linear_reg_dict = {"model": "Linear Regression", 
                   "Mean Absolute Error": mae, 
                   "Root Mean Squared Error": rmse}

In [24]:
# Fit a LASSO regression on the training partition, and then evaluate it on the testing partition
# This will shrink most of the variable coefficients to zero for automated variable selection
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso

linear_reg_with_lasso = linear_model.Lasso(alpha=0.1)
linear_reg_with_lasso.fit(X_train,y_train)

mae = mean_absolute_error(y_test, linear_reg_with_lasso.predict(X_test))
mse = mean_squared_error(y_test, linear_reg_with_lasso.predict(X_test))
rmse = np.sqrt(mse)
lasso_reg_dict = {"model": "LASSO Regression", 
                   "Mean Absolute Error": mae, 
                   "Root Mean Squared Error": rmse}

In [25]:
# Use the LASSO regression to identify the most useful variable to predict the target (i.e., the variables whose coefficient didn't shrink to zero)

variable_names = list(X.columns) 
linear_reg_with_lasso_coef = linear_reg_with_lasso.coef_

for variable, coef in zip(variable_names, linear_reg_with_lasso_coef):
    # if the coefficient didn't shrink to zero
    if (coef != 0):
        # if it's a categorical variable
        if variable in new_categorical_variables:
            variable_substrings = variable.split("_")  # split up the new categorical variable name by underscores
            categorical_variable = '_'.join(variable_substrings[:-1])  # retrieve the original categorical variable name (e.g., 'x05a', 'perf_status_0923', etc.)
            category = variable_substrings[-1:][0]  # retrieve the category chosen for the categorical variable (e.g., '1', '10', '2013', 'A', etc.)
            if any(character.isdigit() for character in category):
                category = int(category)
            else:
                category = str(category)
            print(variable, ":", variable_labels_dict[categorical_variable], ":", categorical_variables_categories_dict[variable_formats_dict[categorical_variable]][category], ":", coef)
        # else it's a numeric variable
        else:
            print(variable, ":", variable_labels_dict[variable], ":", coef)

x74r : Age at last birthday | Respondent : 0.0005843405861712142
term : Mortgage Term (in Years) at Origination : -0.0045401124003684665
ltv : Mortgage Loan-to-Value Ratio at Origination (Percent) : -0.0005094868536450399
dti : Mortgage Debt-to-Income (Back End) Ratio at Origination (Percent) : 0.0009266149212613681
pti : Mortgage Payment-to-Income (Front End) Ratio at Origination (Percent) : -0.0012526036648911915
score_orig_r : VantageScore 3.0 at Origination | Respondent : -0.0007694060319659019
score_0317_r : VantageScore 3.0 in March 2017 | Respondent : 7.059434770453949e-05
score_1217_r : VantageScore 3.0 in December 2017 | Respondent : 0.00015113109467252347
score_0918_r : VantageScore 3.0 in September 2018 | Respondent : -0.0001869133369847791
score_0619_r : VantageScore 3.0 in June 2019 | Respondent : -0.00016726597936467743
score_0919_r : VantageScore 3.0 in September 2019 | Respondent : -7.940300000700951e-06
score_0621_r : VantageScore 3.0 in June 2021 | Respondent : -1.150

In [26]:
# Fit a Random Forest regression model on the training partition, and then evaluate it on the testing partition
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

params = {'n_estimators': 100, 'max_depth': 4, 'random_state':0}
random_forest_reg = RandomForestRegressor(**params)
random_forest_reg.fit(X_train, y_train)

mae = mean_absolute_error(y_test, random_forest_reg.predict(X_test))
mse = mean_squared_error(y_test, random_forest_reg.predict(X_test))
rmse = np.sqrt(mse)
random_forest_reg_dict = {"model": "Random Forest Regression", 
                          "Mean Absolute Error": mae, 
                          "Root Mean Squared Error": rmse}

In [27]:
# Use the Random Forest regression model to identify the top N most important feature based upon their respective feature importance values

top_N = 20
feature_importances = pd.Series(data=random_forest_reg.feature_importances_, index=X_train.columns)
feature_importances.sort_values(ascending=False, inplace=True)
top_N_features = feature_importances.head(top_N)

# Print the top N feature by importance from Random Forest regression model in descending order
for variable in top_N_features.index:
    feature_importance = top_N_features[variable]
    # if the feature importance is greater than zero
    if (feature_importance > 0):
        # if it's a categorical variable
        if variable in new_categorical_variables:
            variable_substrings = variable.split("_")  # split up the new categorical variable name by underscores
            categorical_variable = '_'.join(variable_substrings[:-1])  # retrieve the original categorical variable name (e.g., 'x05a', 'perf_status_0923', etc.)
            category = variable_substrings[-1:][0]  # retrieve the category chosen for the categorical variable (e.g., '1', '10', '2013', 'A', etc.)
            if any(character.isdigit() for character in category):
                category = int(category)
            else:
                category = str(category)
            print(variable, ":", variable_labels_dict[categorical_variable], ":", categorical_variables_categories_dict[variable_formats_dict[categorical_variable]][category], ":", feature_importance)
        # else it's a numeric variable
        else:
            print(variable, ":", variable_labels_dict[variable], ":", feature_importance)

x60_2 : Which one of the following best describes this property? : Mobile home or manufactured home : 0.18077040459346722
pmms : Freddie Macs Primary Mortgage Market Survey (PMMS) Rate at Origination (Percent) : 0.11390027784364364
loan_amount_cat_1 : Mortgage Loan Amount at Origination (Categorical) : Less than $50,000 : 0.10544147550889055
term : Mortgage Term (in Years) at Origination : 0.09562646685749267
x66_5 : Which one of the following best describes how you use this property? : Rental or investment property : 0.05962129690771107
score_orig_r : VantageScore 3.0 at Origination | Respondent : 0.036895231137024045
x44_1 : Does this mortgage have... | An adjustable rate (one that can change over the life of the loan)? : Yes : 0.030601390165239807
loan_amount_cat_2 : Mortgage Loan Amount at Origination (Categorical) : $50,000 to  $99,999 : 0.026592875928340552
perf_status_1216_1 : Mortgage Performance Status in December 2016 : 30 to  59  days past due date : 0.019886533893666694
mtm

In [28]:
# Compare the performance metrics for the various models on the holdout testing data

all_model_performance_metrics = [grad_boost_reg_dict,
                                linear_reg_dict,
                                lasso_reg_dict,
                                random_forest_reg_dict]

all_model_performance_metrics_df = pd.DataFrame(all_model_performance_metrics)
print(all_model_performance_metrics_df)

                          model  Mean Absolute Error  Root Mean Squared Error
0  Gradient Boosting Regression             0.218028                 0.428093
1             Linear Regression             0.246813                 0.431542
2              LASSO Regression             0.230288                 0.455678
3      Random Forest Regression             0.221772                 0.426914
